In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns

import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical

In [ ]:
path = '/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/'
os.listdir(path)

In [ ]:


data = pd.read_csv(path+'icml_face_data.csv')
data


In [ ]:
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(20,5))
sns.countplot(data = data[data[' Usage']=='Training'], x='emotion', ax=ax1).set_title('Training')
ax1.set_xticklabels(emotions.values())
sns.countplot(data = data[data[' Usage']=='PublicTest'], x='emotion', ax=ax2).set_title('Testing')
ax2.set_xticklabels(emotions.values())
sns.countplot(data = data[data[' Usage']=='PrivateTest'], x='emotion', ax=ax3).set_title('Validation')
ax3.set_xticklabels(emotions.values())

In [ ]:
def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label

  
def sample_plot(x,y=None):
    #x, y are numpy arrays
    n = 20
    samples = random.sample(range(x.shape[0]),n)
    
    fig, axs = plt.subplots(2,10, figsize=(25,5), sharex=True, sharey=True)
    ax = axs.ravel()
    for i in range(n):
        ax[i].imshow(x[samples[i],:,:], cmap=plt.get_cmap('gray'))
        ax[i].set_xticks([])
        ax[i].set_yticks([])
        if y is not None:
            ax[i].set_title(emotions[y[samples[i]]])

In [ ]:
train_image_array, train_image_label = prepare_data(data[data[' Usage']=='Training'])
val_image_array, val_image_label = prepare_data(data[data[' Usage']=='PrivateTest'])
test_image_array, test_image_label = prepare_data(data[data[' Usage']=='PublicTest'])

In [ ]:
train_images = train_image_array.reshape((train_image_array.shape[0], 48, 48, 1))
train_images = train_images.astype('float32')/255
val_images = val_image_array.reshape((val_image_array.shape[0], 48, 48, 1))
val_images = val_images.astype('float32')/255
test_images = test_image_array.reshape((test_image_array.shape[0], 48, 48, 1))
test_images = test_images.astype('float32')/255

In [ ]:
data[' Usage'].value_counts()

In [ ]:
train_labels = to_categorical(train_image_label)
val_labels = to_categorical(val_image_label)
test_labels = to_categorical(test_image_label)

In [ ]:
def plot_all_emotions():
    fig, axs = plt.subplots(1, 7, figsize=(30, 12))
    fig.subplots_adjust(hspace = .2, wspace=.2)
    axs = axs.ravel()
    for i in range(7):
        idx = data[data['emotion']==i].index[i]
        axs[i].imshow(train_images[idx][:,:,0], cmap='gray')
        axs[i].set_title(emotions[train_labels[idx].argmax()])
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])

In [ ]:
plot_all_emotions()

In [ ]:
# Convolutional Neural Network (CNN)

# Se crea un modelo secuencial y se agregan capas a la red neuronal

from keras import models
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:


#50 epochs

history = model.fit(train_images, train_labels,
                    validation_data=(test_images, test_labels),  epochs=50, batch_size=32)



In [ ]:
pred_test_labels = model.predict(test_images)

In [ ]:
from PIL import Image 
import PIL 
from six.moves import urllib

urllib.request.urlretrieve("https://imgc.artprintimages.com/img/print/the-godfather-marlon-brando-1972_u-l-q12pfjf0.jpg?h=550&p=0&w=550", "godfather.png")
urllib.request.urlretrieve("https://www.pilotonline.com/resizer/s81d-dFjGylRCkOBJJmMPuA8_s0=/415x560/top/arc-anglerfish-arc2-prod-tronc.s3.amazonaws.com/public/HNMTBLBEZNFAZOOPRLZWMI6CMY.jpg", "bob.png")
urllib.request.urlretrieve("https://pbs.twimg.com/profile_images/1140172161602195457/czwGGE5y.png", "bready.png")

img = PIL.Image.open("godfather.png")

img

In [ ]:
from keras.preprocessing import image
import cv2
import os

#"/data/haarcascades/haarcascade_frontalface_default.xml"

face_haar_cascade = cv2.CascadeClassifier("/content/haarcascade_frontalface_default.xml")

In [ ]:
from google.colab.patches import cv2_imshow

test_image1 = cv2.imread('/kaggle/working/godfather.png')
test_image2 = cv2.imread('/kaggle/working/bob.png')
test_image3 = cv2.imread('/kaggle/working/bready.png')


In [ ]:
gray_image1 = cv2.cvtColor(test_image1, cv2.COLOR_BGR2GRAY)
gray_image2 = cv2.cvtColor(test_image2, cv2.COLOR_BGR2GRAY)
gray_image3 = cv2.cvtColor(test_image3, cv2.COLOR_BGR2GRAY)

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
from keras.preprocessing.image import img_to_array

In [ ]:
faces = face_cascade.detectMultiScale(gray_image1,1.1,4)

for (x,y,w,h) in faces:
    cv2.rectangle(test_image1,(x,y),(x+w,y+h),(255,0,0))
    roi_gray = gray_image1[y:y+w,x:x+h]
    roi_gray =cv2.resize(roi_gray,(48,48))
    image_pixels = img_to_array(roi_gray)
    image_pixels = np.expand_dims(image_pixels, axis = 0)
    image_pixels /= 255
    predictions = model.predict(image_pixels)
    max_index = np.argmax(predictions[0])
    emotion_detection = ( 'Angry','Disgust','Fear','Happy', 'Sad','Surprise', 'Neutral')
    emotion_prediction = emotion_detection[max_index]
    print(emotion_prediction)
    
    font =cv2.FONT_HERSHEY_SIMPLEX
    org = (50,50)
    fontScale = 1
    color = (0, 253, 0)
    thickness = 2
    image = cv2.putText(test_image1,emotion_prediction, org, font, 
                        fontScale, color, thickness, cv2.LINE_AA)
    cv2_imshow(image)

In [ ]:
faces = face_cascade.detectMultiScale(gray_image2,1.1,4)

for (x,y,w,h) in faces:
    cv2.rectangle(test_image2,(x,y),(x+w,y+h),(255,0,0))
    roi_gray = gray_image2[y:y+w,x:x+h]
    roi_gray =cv2.resize(roi_gray,(48,48))
    image_pixels = img_to_array(roi_gray)
    image_pixels = np.expand_dims(image_pixels, axis = 0)
    image_pixels /= 255
    predictions = model.predict(image_pixels)
    max_index = np.argmax(predictions[0])
    emotion_detection = ( 'Angry','Disgust','Fear','Happy', 'Sad','Surprise', 'Neutral')
    emotion_prediction = emotion_detection[max_index]
    print(emotion_prediction)
    
    font =cv2.FONT_HERSHEY_SIMPLEX
    org = (50,50)
    fontScale = 1
    color = (0, 253, 0)
    thickness = 2
    image = cv2.putText(test_image2,emotion_prediction, org, font, 
                        fontScale, color, thickness, cv2.LINE_AA)
    cv2_imshow(image)

In [ ]:
faces = face_cascade.detectMultiScale(gray_image3,1.1,4)

for (x,y,w,h) in faces:
    cv2.rectangle(test_image3,(x,y),(x+w,y+h),(255,0,0))
    roi_gray = gray_image3[y:y+w,x:x+h]
    roi_gray =cv2.resize(roi_gray,(48,48))
    image_pixels = img_to_array(roi_gray)
    image_pixels = np.expand_dims(image_pixels, axis = 0)
    image_pixels /= 255
    predictions = model.predict(image_pixels)
    max_index = np.argmax(predictions[0])
    emotion_detection = ( 'Angry','Disgust','Fear','Happy', 'Sad','Surprise', 'Neutral')
    emotion_prediction = emotion_detection[max_index]
    print(emotion_prediction)
    
    font =cv2.FONT_HERSHEY_SIMPLEX
    org = (50,50)
    fontScale = 1
    color = (0, 253, 0)
    thickness = 2
    image = cv2.putText(test_image3,emotion_prediction, org, font, 
                        fontScale, color, thickness, cv2.LINE_AA)
    cv2_imshow(image)

In [ ]:
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

conf_mat = confusion_matrix(test_labels.argmax(axis=1), pred_test_labels.argmax(axis=1))

fig, ax = plot_confusion_matrix(conf_mat=conf_mat,
                                show_normed=True,
                                show_absolute=False,
                                class_names=emotions.values(),
                                figsize=(8, 8))
fig.show()

In [ ]:
#!pip install google-colab